In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import openpyxl
import xlrd
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import re

In [3]:
import setuptools

In [4]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [5]:
path_in_excel = '/drive/MyDrive/Doutorado_v2/Staging/'
path_out_excel = '/drive/MyDrive/Doutorado_v2/Resumos_BERT'

In [6]:
a = os.listdir('/drive/MyDrive/Doutorado_v2/Resumos_BERT/')
df = pd.DataFrame(list(a), columns=['name_file'])
df.head()

,name_file
0,file_reportagens_1.xlsx
1,file_reportagens_2.xlsx
2,file_reportagens_3.xlsx
3,file_reportagens_4.xlsx
4,file_reportagens_5.xlsx


In [7]:
df.shape

(35, 1)

In [8]:
for i in range(0, len(df)):
    name_file_df = df.iloc[i]['name_file']
    path_in = '/drive/MyDrive/Doutorado_v2/Resumos_BERT/'
    file_in = os.path.join(path_in, name_file_df)
    df_file_00 = pd.read_excel(file_in)
    if i == 0:
        file_copy = df_file_00
    else:
       file_copy = pd.concat([file_copy, df_file_00])

file_copy.shape

(3225, 10)

In [9]:
file_copy['Sumarization'] = file_copy['Sumarization'].str.strip('[]')

In [10]:
file_copy['Sumarization_Count'] = file_copy['Sumarization'].str.len()

In [11]:
((file_copy['Sumarization_Count'].sum()) / (file_copy['Size_notice'].sum())) * 100

25.088995142138504

In [12]:
file_copy = file_copy.reset_index()

In [13]:
file_copy.columns

Index(['index', 'name_file', 'data_noticia_orig', 'data_noticia_corrig', 'Ano',
       'Publication_orig', 'titulo', 'Size_notice', 'Sumarization',
       'Size_notice_cum', 'text_full', 'Sumarization_Count'],
      dtype='object')

In [14]:
file_copy = file_copy[['name_file', 'data_noticia_orig', 'data_noticia_corrig', 'Ano',
       'Publication_orig', 'titulo', 'Size_notice', 'Size_notice_cum', 'Sumarization', 'Sumarization_Count', 'text_full']]

In [15]:
file_copy.shape

(3225, 11)

In [16]:
file_copy.iloc[:, 0:12].isnull().sum()

name_file              0
data_noticia_orig      0
data_noticia_corrig    0
Ano                    0
Publication_orig       0
titulo                 0
Size_notice            0
Size_notice_cum        0
Sumarization           1
Sumarization_Count     1
text_full              0
dtype: int64

In [17]:
file_copy = file_copy[file_copy['Sumarization'].notna()]

In [18]:
file_copy.shape

(3224, 11)

In [19]:
file_copy.head()

,name_file,data_noticia_orig,data_noticia_corrig,Ano,Publication_orig,titulo,Size_notice,Size_notice_cum,Sumarization,Sumarization_Count,text_full
0,3809.pdf,"March 10, 2020 (10:22)",2020-03-10,2020,Jornal O Tempo - Cities,Copam julga recurso da Igreja contra retorno d...,77,77,Polêmica se arrasta há mais de um ano; empresa...,75.0,b
1,2749.pdf,"June 16, 2020 (03:00)",2020-06-16,2020,Jornal O Tempo - Cities,Justiça dá 15 dias para prefeitura se posicion...,92,169,Moradores de povoado temem contaminação das ág...,90.0,b
2,3635.pdf,"January 30, 2020 (14:09)",2020-01-30,2020,Zero Hora Field and Crop,Como aproveitar os benefícios do coco durante ...,97,266,Além de matar a sede a fruta tropical mantém h...,95.0,b
3,2849.pdf,"June 30, 2020 (23:14)",2020-06-30,2020,Jornal O Tempo - Cities,"Barragens da Vale em Minas podem se romper, e ...",99,365,Cinco estruturas da mineradora em Ouro Preto e...,97.0,b
4,3594.pdf,"July 25, 2017",2017-07-25,2017,DCI - News,Mineração,102,467,Órgão anti-fraudes da Inglaterra abriu investi...,100.0,b


In [20]:
file_copy.tail()

,name_file,data_noticia_orig,data_noticia_corrig,Ano,Publication_orig,titulo,Size_notice,Size_notice_cum,Sumarization,Sumarization_Count,text_full
3220,1199.pdf,"February 06, 2019 (06:00)",2019-02-06,2019,InfoMoney - Markets,Piora na legislação foi emblemática para tragé...,19964,75393,SÃO PAULO - Doze dias após o rompimento da bar...,3811.0,b
3221,1901.pdf,"January 18, 2021 (08:13)",2021-01-18,2021,InfoMoney - Markets,"Movida compra gestora de frotas, Petrobras que...",21650,97043,SÃO PAULO - O noticiário corporativo tem como ...,4332.0,b
3222,1900.pdf,"January 18, 2021 (10:36)",2021-01-18,2021,InfoMoney - Markets,Ações de CVC e aéreas sobem; Vale e siderúrgic...,22020,119063,SÃO PAULO - A sessão desta segunda-feira (18) ...,3856.0,b
3223,1397.pdf,"March 27, 2014 (08:46)",2014-03-27,2014,Fator Brasil - Steel,"MMX Mineração com prejuízo de R$353,1 milhões ...",33792,152855,2014 segue com grandes desafios mas também com...,6053.0,b
3224,1424.pdf,"May 09, 2014 (07:26)",2014-05-09,2014,Fator Brasil - Steel,Desafios e perspectivas da indústria de minera...,48884,201739,Além de contar com uma das maiores reservas mi...,9597.0,b


In [21]:
file_copy.to_excel(r'/drive/MyDrive/Doutorado_v2/E02_file_reportagens.xlsx', index=False)